#### A simple model of the hardware: a chain of $n$ simple repeaters

    r[1] ---- r[2] ---- r[3] ---- ... ---- r[n]
    
A simple repeater: a repeater that can support 2 links that both have the same lifetime; and has ideal gates (applying gates on the qubits does not affect lifetimes).

In [18]:
from threading import Timer
import time
import random
import math
from IPython.display import clear_output


# Helper function: flips an unfair coin.
def flip_unfair_coin(p):
    return 'H' if random.random() < p else 'T'
    
    
class simple_repeater(object):
    # Constructor.
    def __init__(self, id, link_lifetime = 1, link_creation_time = 0.005, parent_chain = None):
        # The repeater id.
        self.id = id
        # The lifetime of the links, specifically the time in which right link expires
        self.link_lifetime = link_lifetime
        # The time needed to establish a link.
        self.link_creation_time = link_creation_time
        # The probability that a link is create successfully.
        self.link_creation_prob = 1
        # Repeaters/nodes to the right and to the left.
        # They are simple_repeater objects.
        # None means the link is not alive.
        self.rightNode = None
        self.leftNode  = None
        # Timers to keep track of the alive time of each link.
        self.rightLinkTimer = None
        self.leftLinkTimer = None
        self.rightLinkTimerStart=None
        # Repeater chain in which the repeater is embedded.
        self.parent_chain = parent_chain
        # The repeater's queued tasks.
#         self.task_queue = []

    # Link repeater to another one sitting somewhere on its right.
    def create_right_link_helper(self, other,expire_time=None):
        self.rightNode = other
        other.leftNode = self
        # Start expiry timer.
        if expire_time is not None:
          self.rightLinkTimer = Timer(expire_time, self.right_link_expires)
          self.link_lifetime=expire_time
        else:
          self.rightLinkTimer = Timer(self.link_lifetime, self.right_link_expires)
        self.rightLinkTimerStart=time.time()
        self.rightLinkTimer.start()
        if self.parent_chain.GUI_enabled:
            self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
        else:
            print("link created between " + self.id + " and " + self.rightNode.id)
            # Check for success.
            if self.parent_chain.success():
               print("Success!") 
            
    def create_right_link(self, other):
        # Creating a link is probabilistic.
        if flip_unfair_coin(self.link_creation_prob) == 'H':
            # And takes time.
            Timer(self.link_creation_time, self.create_right_link_helper, [other]).start()
    
#     # Link repeater to another one sitting somewhere on its left.
#     def create_left_link_helper(self, other):
#         self.leftNode = other
#         other.rightNode = self
#         self.leftLinkTimer = Timer(self.link_lifetime, self.left_link_expires)
#         self.leftLinkTimer.start()
#         if self.parent_chain.GUI is not None:
#             self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
            
#     def create_left_link(self, other):
#         if flip_unfair_coin(self.link_creation_prob) == 'H':
#             Timer(self.link_creation_time, self.create_left_link_helper, [other]).start()
        
    # Right link dies.
    def right_link_expires(self):
        if self.rightNode is not None:
            self.rightNode.leftNode = None
            self.rightNode = None
            self.rightLinkTimer = None
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
        
    # Left link dies.
    def left_link_expires(self):
        if self.leftNode is not None:
            self.leftNode.rightNode = None
            self.leftNode = None
            self.leftLinkTimer = None
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())

    # Perform entanglement swap.
    def swap(self):
#         print("Swapping in repeater", self.id)
        if self.leftNode is not None and self.rightNode is not None:
            #time in which self.leftNode--self will expire, ie the qubits will decohere
            left_qb_expiry=self.leftNode.link_lifetime-(time.time()-self.leftNode.rightLinkTimerStart)
            #time in which self--self.rightNode will expire, ie the qubits will decohere
            right_qb_expiry=self.link_lifetime-(time.time()-self.rightLinkTimerStart)
            #the connection self.leftNode--self.rightNode will expire in the min of the above times
            connection_expiry=min(left_qb_expiry,right_qb_expiry)
            self.leftNode.create_right_link_helper(self.rightNode,connection_expiry)
            # Change link states to not alive.
            self.rightNode = None
            self.leftNode  = None
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
            
     

class simple_repeater_chain(object):
    def __init__(self, length = 5, link_lifetime = 1, link_creation_time = 0.005,
                 GUI_enabled = False, repeater_type=simple_repeater):
        self.length = length
        self.link_lifetime = link_lifetime
        self.link_creation_time = link_creation_time
        self.link_creation_prob = 1
        # Initialize the repeaters.
        self.repeaters = []
        for i in range(self.length):
            new_repeater = repeater_type(str(i), link_lifetime, link_creation_time, self)
            self.repeaters.append(new_repeater)
        # Initialize the GUI.
        self.GUI_enabled = GUI_enabled
        if self.GUI_enabled:
            self.GUI = GUI()
#             self.GUI.draw_state(self.pack_snapshot())
            
    def success(self):
        if self.repeaters[0].rightNode is None:
            return False
        return self.repeaters[0].rightNode.id == self.repeaters[-1].id

    # Says which link each link is connected to.
    def pack_snapshot(self):
        snapshot = []  
        for repeater in self.repeaters:
            if repeater.rightNode is not None:
                snapshot.append(repeater.rightNode.id)
            else:
                # -1 means it's not linked to any repeaters.
                snapshot.append(-1)
#         print(snapshot)
        return snapshot

    

# GUI (not finished yet)
class GUI(object):
    def __init__(self):
        self.request_queue = []
        self.busy = False
        
    def draw_state(self, state):
        t1 = ""
        t2 = ""
        s = ""
        for i in range(len(state)):
#             s = s + "r" + "[" + str(i+1) + "]"
            s = s + "r"
            if i < len(state) - 1:
                if int(state[i]) == i + 1:
                    s += " ---- "
                    t1 += "  " + "    " + " "
                    t2 += "  " + "    " + " "
                else:
                    s += "      "
                    if state[i] != -1:
                        t1 += " /" + (int(state[i])-i-1) * ("    " + "   ") + "    " + "\\"
                        t2 += "  " + (int(state[i])-i-1) * ("----" + "---") + "----" + " "
                    else: 
                        t1 += "  " + "    " + " "
                        t2 += "  " + "    " + " "
        if int(state[0]) == len(state)-1:
            s += "       Success!"
        clear_output(wait = True)
        print(t2)
        print(t1)
        print(s)
        
    def update_gui(self, new_state):
        self.request_queue.append(new_state)
        if self.busy == False:
            self.get_busy()
        
    def get_busy(self):
        self.busy = True
        while len(self.request_queue) > 0:
            new_state = self.request_queue.pop(0)
            self.draw_state(new_state)
        self.busy = False 

In [13]:
class repeater_v2(simple_repeater):
  def __init__(self, id, link_lifetime = 1, link_creation_time = 0.005, parent_chain = None):
    
    super().__init__(id, link_lifetime, link_creation_time, 
                      parent_chain)

  def create_right_link_helper(self, other,expire_time=None):
      # Creating a link is probabilistic. returns True if successful.
      should_create = flip_unfair_coin(self.link_creation_prob) == 'H'
      if should_create:
        super().create_right_link_helper(other,expire_time)
        return True
      else:
        return False 
  
  def create_right_link(self, other, retry=True, retry_count=3):
      retry_count = retry_count if retry else 1
      for _ in range(retry_count):
          # Creating link takes time.
          if self.rightNode is None:
            success = self.create_right_link_helper(other)
            if success:
              return
            time.sleep(self.link_creation_time)
          self.parent_chain.indicate_link_failiure([self, other])
    
  # Right link dies.
  def right_link_expires(self):
    if self.rightNode is not None:
      other = self.rightNode
      super().right_link_expires()
      # self.parent_chain.indicate_link_failiure([self, other]) // yet to decide if the link expires, will it be same as link failiure?
        
  # Left link dies.
  def left_link_expires(self):
    if self.leftNode is not None:
      other = self.leftNode
      super().left_link_expires()
        # self.parent_chain.indicate_link_failiure([self, other]) 


class repeater_v2_chain(simple_repeater_chain):
  STATE_INIT, STATE_ACTIVE, STATE_BROKEN = 'INIT', 'ACTIVE', 'BROKE' 
  
  def __init__(self, length = 5, link_lifetime = 1, link_creation_time = 0.005, GUI_enabled = False):
    super().__init__(length, link_lifetime, link_creation_time, GUI_enabled, repeater_type=repeater_v2)

    # self.failed_links = [] #In future version of this protocol we can make use of the failed links to re-route.
    self.state = self.STATE_INIT


  def indicate_link_failiure(self, repeaters):
      """Repeaters should call this function to indicate link creation failiure.
       We can add broadcast function in future to inform all nodes that they can free resources since complete link was not established. 
      """
      # self.failed_links.append(repeaters)
      self.state = self.STATE_BROKEN
      print(f'Link creation failiure at: {repeaters[0].id}-{repeaters[1].id}.')

  def make_chain_active(self):
    if self.state != self.STATE_BROKEN:
      self.state = self.STATE_ACTIVE

  def is_active(self):
    return self.state == self.STATE_ACTIVE

#### Our goal is to end up with the first and last repeaters --- they are really nodes --- in the chain entangled.

Here's a possible protocol.


#### Protocol 1:  a naive protocol

In [14]:
# Try once to establish all links, starting from the leftmost repeater. Then swap them one by one.
def protocol1(simple_repeater_chain):
    # Create links
    for i in range(len(simple_repeater_chain.repeaters)-1):
        this_repeater = simple_repeater_chain.repeaters[i]
        next_repeater = simple_repeater_chain.repeaters[i+1]
        this_repeater.create_right_link(next_repeater)
        # Wait a bit
        time.sleep(this_repeater.link_creation_time)
    # Perform swap.
    for repeater in simple_repeater_chain.repeaters:
        repeater.swap()
        # Wait a bit
        time.sleep(repeater.link_creation_time)

#### Let's test it
Play with the parameters and see what happens!

In [15]:
chain = simple_repeater_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5, 
                              GUI_enabled = True)


protocol1(chain)

                                                 
                                                 
r      r      r      r      r      r      r      r


#### Protocol 2
Here is an alternate protocol that attempts to establish connection between nodes 0 and i+1 as soon as connection between 0 and i is established. This ensures that the first established link does not expire by the time connection between the last two nodes is established

In [16]:
def protocol2(simple_repeater_chain):
  for i in range(len(simple_repeater_chain.repeaters)-1):
    this_repeater = simple_repeater_chain.repeaters[i]
    next_repeater = simple_repeater_chain.repeaters[i+1]
    # print("creating right link from i=",i)
    this_repeater.create_right_link(next_repeater)
    # Wait a bit
    time.sleep(this_repeater.link_creation_time)
    #perform swap if it is not the first node
    if(i!=0):
      # print("doing swap at i=",i)
      this_repeater.swap()
      # Wait a bit
      time.sleep(this_repeater.link_creation_time)

In [17]:
chain = simple_repeater_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5,
                              GUI_enabled = True)


protocol2(chain)

                                                 
                                                 
r      r      r      r      r      r      r      r


#### Protocol 3: link spawns in the middle of the chain

It is not yet clear whether this protocol does anything better than the other ones. 

This because the it is not yet clear how the swaps affect the lifetime of the link.

But if we assuming that swaps do not affect the link lifetime, and that the links expire only when one of the qubits supporting it decoheres, then this protocol should perform better --- because no qubit supports the link for too long.

This is in contrast with, say, in Protocol 1 above. In Protocol 1 the very first repeater supports the link from the very beginning and until the whole chain has been created --- for long chains the qubit in the first repeated can't hold the link long enough for the entire chain to be created.

In [64]:
def protocol3(simple_repeater_chain):
    middle = math.floor(len(simple_repeater_chain.repeaters) / 2)
    # create the middle link
    this_repeater = simple_repeater_chain.repeaters[middle]
    next_repeater = simple_repeater_chain.repeaters[middle + 1]
    this_repeater.create_right_link(next_repeater)
    # Wait a bit
    time.sleep(this_repeater.link_creation_time)
    # step to the sides
    i = middle - 1
    j = middle + 1
    while True:
        if i > -1:
            # create links on the left
            this_repeater = simple_repeater_chain.repeaters[i]
            next_repeater = simple_repeater_chain.repeaters[i + 1]
            this_repeater.create_right_link(next_repeater)
        if j < len(simple_repeater_chain.repeaters):
            # create links on the right
            this_repeater = simple_repeater_chain.repeaters[j - 1]
            next_repeater = simple_repeater_chain.repeaters[j]
            this_repeater.create_right_link(next_repeater)
        # wait a bit
        time.sleep(2 * this_repeater.link_creation_time)
        # swap
        simple_repeater_chain.repeaters[i + 1].swap()
        simple_repeater_chain.repeaters[j - 1].swap()
        # move to next repeaters  
        i -= 1
        j += 1
        if i < 0 and j > len(simple_repeater_chain.repeaters) - 1:
            break

In [69]:
chain = simple_repeater_chain(length = 13, link_lifetime = 5, link_creation_time = 0.5, 
                              GUI_enabled = True)


protocol3(chain)

                                                                                    
                                                                                    
r      r      r      r      r      r      r      r      r      r      r      r      r


#### Protocol 1.1

This protocol improves upon Protocol 1 by:
* Only allowing swapping if the complete chain is ready.
* Inform all the repeaters at once to create link. 

##### Protocol definition
* Inform all the nodes at once which node they need to link with (can be done classically) and then wait for the time it would take maximum possible retries to complete.

* Incase link creation fails, repeaters retry link creation. If all attempts for link creation fail, repeaters can inform parent chain/broadcast that the link creation  failed or link expired. If Parent chain receives information on link creation failiure, link is considered in broken state.

* If no error was received until the (retries allowed * link_creation_time) seconds, the link is made active.

* Start `swap()` operations  and keep swapping if the chain is in Active State.

(PS. please use this protocol with GUI_enabled=False, does not work well with GUI)
 

In [10]:
def protocol1_1(repeater_chain):
    # Create links
    retries = 3
    for i in range(len(repeater_chain.repeaters)-1):
        # Inform all the repeaters simaltaneously whom they'll be linking up with.
        this_repeater = repeater_chain.repeaters[i]
        next_repeater = repeater_chain.repeaters[i+1]
        this_repeater.create_right_link(next_repeater, retry_count=retries)

    # Wait for a repeaters to indicate to `Parent Chain` if they failed in creating the link.
    time.sleep(repeater_chain.link_creation_time * retries)
    repeater_chain.make_chain_active() # If no failiures, make active.
    
    # Perform swap.
    for repeater in repeater_chain.repeaters:
        if repeater_chain.is_active():
          repeater.swap()
          # Wait a bit
          time.sleep(repeater.link_creation_time)

In [11]:
chain = repeater_v2_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5, GUI_enabled = False, )
protocol1_1(chain)

link created between 0 and 1
link created between 1 and 2
link created between 2 and 3
link created between 3 and 4
link created between 4 and 5
link created between 5 and 6
link created between 6 and 7
link created between 0 and 2
link created between 0 and 3
link created between 0 and 4
link created between 0 and 5
link created between 0 and 6
link created between 0 and 7
Success!
